# Лабораторная работа 3. Pandas, метод ближайших соседей и решающие деревья.

Name: Dinara Sadykova

Student ID: 16BD02108

Email: madendinara@gmail.com

In [12]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [38]:
data = pd.read_csv("2008.csv")
data.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,7009728.0,7.009728e+06,7.009728e+06,7.009728e+06,6.873482e+06,7.009728e+06,6.858079e+06,7.009728e+06,7.009728e+06,6.855029e+06,...,7.009728e+06,6.858079e+06,6.872670e+06,7.009728e+06,7.009728e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06,1.524735e+06
mean,2008.0,6.375130e+00,1.572801e+01,3.924182e+00,1.333830e+03,1.326086e+03,1.481258e+03,1.494801e+03,2.224200e+03,1.273224e+02,...,7.263870e+02,6.860852e+00,1.645305e+01,1.960618e-02,2.463006e-03,1.577206e+01,3.039031e+00,1.716462e+01,7.497434e-02,2.077098e+01
std,0.0,3.406737e+00,8.797068e+00,1.988259e+00,4.780689e+02,4.642509e+02,5.052251e+02,4.826728e+02,1.961716e+03,7.018731e+01,...,5.621018e+02,4.933649e+00,1.133280e+01,1.386426e-01,4.956753e-02,4.009912e+01,1.950287e+01,3.189495e+01,1.837940e+00,3.925964e+01
min,2008.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.200000e+01,...,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2008.0,3.000000e+00,8.000000e+00,2.000000e+00,9.280000e+02,9.250000e+02,1.107000e+03,1.115000e+03,6.220000e+02,7.700000e+01,...,3.250000e+02,4.000000e+00,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2008.0,6.000000e+00,1.600000e+01,4.000000e+00,1.325000e+03,1.320000e+03,1.512000e+03,1.517000e+03,1.571000e+03,1.100000e+02,...,5.810000e+02,6.000000e+00,1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,0.000000e+00,0.000000e+00
75%,2008.0,9.000000e+00,2.300000e+01,6.000000e+00,1.728000e+03,1.715000e+03,1.909000e+03,1.907000e+03,3.518000e+03,1.570000e+02,...,9.540000e+02,8.000000e+00,1.900000e+01,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,2.100000e+01,0.000000e+00,2.600000e+01
max,2008.0,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.359000e+03,2.400000e+03,2.400000e+03,9.743000e+03,1.379000e+03,...,4.962000e+03,3.080000e+02,4.290000e+02,1.000000e+00,1.000000e+00,2.436000e+03,1.352000e+03,1.357000e+03,3.920000e+02,1.316000e+03


In [39]:
data['CancellationCode'].value_counts()[0]

54904

In [40]:
data['Distance'].mean()

726.3870294253928

In [41]:
data['Distance'].min()

11

In [42]:
data['Distance'].max()

4962

In [43]:
data[data['Distance'] == data['Distance'].min()]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2547298,2008,5,15,4,2037.0,1800,2125.0,1900,OH,4988,...,10.0,7.0,0,NaN,0,145.0,0.0,0.0,0.0,0.0
4392215,2008,8,10,7,1315.0,1220,1415.0,1320,OH,5572,...,8.0,38.0,0,NaN,0,55.0,0.0,0.0,0.0,0.0


In [47]:
data[data.FlightNum == 4988]['Distance'].values

array([834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834,  96, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,
       834, 834, 834, 834, 834, 834, 834, 834, 834, 834, 834,  11, 834,
       834, 834, 414, 223, 223, 223, 223, 223, 223, 223, 223, 223, 223,
       223, 712, 712, 712, 712, 321, 321, 321, 321, 321, 321, 321, 321,
       321, 321, 321, 321, 321, 321, 321, 321, 321, 321, 903, 903, 903,
       903, 903, 903, 903, 903, 903, 903, 903, 903, 903, 903, 90

In [48]:
data[data.FlightNum == 5572]['Distance'].values

array([544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544,
       544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544,
       544, 544, 544, 544, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544,
       544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544,
       544, 544, 544, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329, 329,
       329, 329, 329, 329, 329, 544, 544, 544, 544, 544, 544, 544, 544,
       544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 544, 54

In [49]:
data["Origin"].value_counts().index[0]

'ATL'

In [50]:
data.groupby("Origin")['AirTime'].mean()

Origin
ABE     89.057518
ABI     36.789977
ABQ     91.657166
ABY     35.379759
ACK     50.692124
ACT     29.875977
ACV     58.850658
ACY    108.848214
ADK    151.423913
ADQ     41.987934
AEX     68.543556
AGS     36.933814
AKN     37.321429
ALB    101.860527
ALO     36.373134
AMA     56.196001
ANC    156.163749
ASE     48.307996
ATL     93.320419
ATW     50.696326
AUS    101.266600
AVL     69.899021
AVP     94.665204
AZO     34.846602
BDL    118.701612
BET     57.597815
BFL     64.024511
BGM     66.582260
BGR     90.211095
BHM     82.290713
          ...    
SPS     29.490040
SRQ    103.083807
STL     99.294409
STT    168.681929
STX    168.101176
SUN     48.221851
SUX     44.918919
SWF    132.768826
SYR     84.524087
TEX     93.124183
TLH     59.293080
TOL     48.295047
TPA    117.917963
TRI     47.871046
TUL     80.876802
TUP     42.900000
TUS     92.801864
TVC     49.328770
TWF     41.316563
TXK     43.796312
TYR     30.372715
TYS     81.030894
VLD     45.158351
VPS     75.231116
WRG

In [55]:
res = data.groupby('Origin').AirTime.mean()
res.nlargest(1)

Origin
SJU    203.850843
Name: AirTime, dtype: float64

In [59]:
res = data.groupby('Origin')['DepDelay']
filt = data[data.DepDelay > 0].groupby('Origin')['DepDelay']
(filt.count() / res.count() * (res.count() >= 1000)).idxmax()

'DAL'

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [28]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [29]:
data.shape

(32769, 10)

In [30]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [60]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

Year 1
Month 12
DayofMonth 31
DayOfWeek 7
DepTime 1441
CRSDepTime 1217
ArrTime 1441
CRSArrTime 1378
UniqueCarrier 20
FlightNum 7539
TailNum 5374
ActualElapsedTime 690
CRSElapsedTime 526
AirTime 664
ArrDelay 1154
DepDelay 1135
Origin 303
Dest 304
Distance 1435
TaxiIn 191
TaxiOut 343
Cancelled 2
CancellationCode 5
Diverted 2
CarrierDelay 985
WeatherDelay 600
NASDelay 575
SecurityDelay 157
LateAircraftDelay 565


In [61]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], 
                                                    test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

Здесь вы можете поделиться своими мыслями о задании.

In [ ]:
:)

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.

In [ ]:
несмотря ни на что